In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


Building a Customer Support Agent Foundation with Gemini

**Use Case / Problem:**

Businesses need effective customer support that is responsive, accurate, and
available. Traditional methods face challenges:
- Human agents can be costly and have limited working hours.
- Basic rule-based chatbots lack natural conversation skills and cannot handle
  complex or unforeseen queries.
- Support agents often need access to real-time data (like order status or
  product availability) which simple bots cannot retrieve.
- Analyzing and routing customer requests efficiently can be time-consuming.

**How Generative AI (Gemini) Can Solve This:**

Large Language Models like Google's Gemini offer powerful capabilities to
address these challenges:
1.  **Natural Language Understanding:** They can understand customer queries phrased
    in everyday language.
2.  **Persona & Tone Control (Few-Shot Prompting):** We can guide the AI to adopt
    a specific persona (e.g., a helpful agent for a particular brand) and
    response style by providing examples.
3.  **Accessing External Tools (Function Calling):** The AI can be empowered to
    use external functions or APIs to fetch real-time data (like checking an
    order database) or perform actions.
4.  **Structured Data Generation (JSON Mode):** The AI can output information
    in a structured format (like JSON) for easier processing, logging, or
    routing by other systems.

**Goal of this Notebook:**

This notebook demonstrates how to use the Google Gemini API and Python to
build the foundational components of a customer support agent. We will showcase

**three key Gen AI capabilities:**
* Few-Shot Prompting (for persona)
* Function Calling (for data retrieval)
* Structured Output (for request categorization)


### 1. Setup: Install Libraries, Configure API Key (from Kaggle Secrets), and Define Mock Tools


In [ ]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages

!pip install -U -q "google-genai==1.7.0"


In [35]:
import google.generativeai as genai
import os
import json

genai.__version__

'0.8.3'

**Import Kaggle Secrets client and retrieve the API key**

In [36]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


# --- Mock Functions ---

Define mock (simulated) functions for demonstration purposes. In a real application, the support agent needs to interact with external systems (databases, APIs). These mock functions simulate that interaction, allowing us to test the Function Calling capability without needing actual databases set up for this demo.


In [37]:
def get_order_status(order_id: str) -> dict:
    """MOCK FUNCTION: Simulates looking up order status."""
    print(f"--- MOCK FUNCTION CALLED: get_order_status(order_id='{order_id}') ---")
    mock_db = {
        "ORD12345": {"status": "Shipped", "estimated_delivery": "2024-03-15", "carrier": "ExpressPost"},
        "ORD67890": {"status": "Processing", "estimated_delivery": "2024-03-18", "carrier": "Standard"},
        "ORD11223": {"status": "Delivered", "estimated_delivery": "2024-03-10", "carrier": "Local Courier"}
    }
    status_info = mock_db.get(order_id, {"status": "Not Found", "error": "Invalid order ID."})
    print(f"--- MOCK FUNCTION RETURNED: {status_info} ---")
    return status_info

def find_product_info(product_name: str) -> dict:
    """MOCK FUNCTION: Simulates looking up product info."""
    print(f"--- MOCK FUNCTION CALLED: find_product_info(product_name='{product_name}') ---")
    mock_catalog = {
        "photon blaster x1": {"price": 199.99, "availability": "In Stock", "features": ["Laser targeting", "Rechargeable battery", "Adjustable power levels"]},
        "gravity boots": {"price": 89.50, "availability": "Ships in 3-5 days", "features": ["Magnetic soles", "Comfort fit", "Sizes S/M/L"]},
        "warp drive repair kit": {"price": 450.00, "availability": "In Stock", "features": ["Universal compatibility", "Includes plasma wrench", "DIY instructions"]}
    }
    found_product = None
    for name, info in mock_catalog.items():
        if product_name.lower() in name.lower():
            found_product = info
            break
    if found_product:
        result = found_product
    else:
        result = {"availability": "Not Found", "error": f"Product '{product_name}' not found in catalog."}
    print(f"--- MOCK FUNCTION RETURNED: {result} ---")
    return result

print("Mock functions defined.")

Mock functions defined.


Map function names (for API schema) to actual Python functions.

In [58]:
available_functions = {
    "get_order_status": get_order_status,
    "find_product_info": find_product_info,
}

# 2. Capability Demo: Few-Shot Prompting (Establishing Agent Persona)


* **Problem:** Ensuring the AI agent communicates with the desired brand voice and helpfulness, rather than as a generic AI.
* **Gen AI Solution:** Providing example interactions (few shots) in the initial chat history guides the model's tone and style.




In [59]:
print("\n--- Capability Demo: Few-Shot Prompting ---")
print("Goal: Define the agent's helpful persona for 'GadgetGalaxy'.")


--- Capability Demo: Few-Shot Prompting ---
Goal: Define the agent's helpful persona for 'GadgetGalaxy'.


Define a list of dictionaries representing example user/agent turns. This history serves as the few-shot examples to prime the model's persona.

In [60]:
initial_chat_history = [
    {"role": "user", "parts": ["Hi there!"]},
    {"role": "model", "parts": ["Hello! Welcome to GadgetGalaxy support. How can I help you today?"]},
    {"role": "user", "parts": ["Do you sell gravity boots?"]},
    {"role": "model", "parts": ["Yes, we do! Our Gravity Boots are quite popular. Would you like to know more about their features, price, or availability?"]},
    {"role": "user", "parts": ["I want to check on my recent order."]},
    {"role": "model", "parts": ["I can certainly help with that! Could you please provide your order ID?"]}
]


Initialize a Gemini model instance to create the AI model object we will interact with.
* **Note:** If the model name is invalid or API access fails, this will raise an error.

In [61]:
model_for_few_shot = genai.GenerativeModel('gemini-1.5-flash-latest')


**Start a chat session using the initial history to Primes the specific chat instance with the desired persona.**

In [62]:
chat_session_few_shot = model_for_few_shot.start_chat(history=initial_chat_history)
print("Chat session for few-shot demo initialized with persona history.")

print("\nTesting the persona with a new query...")

Chat session for few-shot demo initialized with persona history.

Testing the persona with a new query...


**Send a new message to test the persona.**

In [63]:
prompt1 = "I need help with my Photon Blaster X1"
print(f"User: {prompt1}")

User: I need help with my Photon Blaster X1


**Get and print the model's response and Observe if the response follows the helpful, brand-aware style from the examples.**

In [64]:
response1 = chat_session_few_shot.send_message(prompt1)
print(f"Agent (Few-Shot): {response1.text}")
print("-" * 30)


Agent (Few-Shot): Okay, I can help with that. To best assist you, could you please tell me what issue you're experiencing with your Photon Blaster X1?  The more details you can provide (e.g., error messages, specific symptoms, what you've already tried), the better I can help you troubleshoot the problem.

------------------------------


In [65]:
prompt2 = "Tell me about your return policy."
print(f"User: {prompt1}")
response2 = chat_session_few_shot.send_message(prompt2)
print(f"Agent (Few-Shot): {response2.text}")
print("-" * 20)

User: I need help with my Photon Blaster X1
Agent (Few-Shot): Our return policy allows for returns within 30 days of purchase for most items.  Items must be in their original packaging and in new, unused condition.  A few exceptions apply, such as opened software or consumables.  We do not accept returns on items damaged due to misuse.  For a full refund, the item must be returned in its original condition.  Shipping costs for returns are generally the responsibility of the customer, unless the return is due to our error.  Before returning an item, please contact our customer service department to obtain a Return Merchandise Authorization (RMA) number.  You can find more details on our website's "Returns and Refunds" page.

--------------------



# 3. Capability Demo: Function Calling (Accessing External Tools)


* **Problem:** Enabling the AI agent to access real-time, external information (like order status) or perform actions (like booking).
* **Gen AI Solution:** Defining 'tools' (our Python functions) that the model can request to call when needed to fulfill a user's request.




In [66]:
print("\n--- Capability Demo: Function Calling ---")
print("Goal: Define and enable tools for fetching order status and product info.")


--- Capability Demo: Function Calling ---
Goal: Define and enable tools for fetching order status and product info.


**Define the structure (schema)** of the available tools for the API to tell the Gemini model about the functions: their names, purposes (description), and expected inputs (parameters).
Map the function names defined in the tools schema to our actual Python functions. It Allows the code to execute the correct Python function when the model requests it by name.


In [67]:
tools_schema = [
    { # Representing a single "Tool" implicitly containing function declarations
        "function_declarations": [
            {
                "name": "get_order_status", # Must match a key in available_functions
                "description": "Retrieves the current status and estimated delivery for a given GadgetGalaxy order ID.",
                # Define parameters using OpenAPI schema format (as dictionaries)
                "parameters": {
                    "type": "object", # Use "object" for OpenAPI schema
                    "properties": {
                        "order_id": {
                            "type": "string", # Use "string" for OpenAPI schema
                            "description": "The unique identifier for the customer's order (e.g., ORD12345)."
                        }
                    },
                    "required": ["order_id"]
                }
            },
            {
                "name": "find_product_info", # Must match a key in available_functions
                "description": "Looks up details about a specific GadgetGalaxy product, such as price, availability, and features.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "product_name": {
                            "type": "string",
                            "description": "The name of the product the customer is asking about (e.g., 'Photon Blaster X1', 'gravity boots'). Should be specific."
                        }
                    },
                    "required": ["product_name"]
                }
            }
        ]
    }
]
# --- END OF CHANGE ---


print("Tool schema defined for Gemini API using dictionary format.")

model_with_tools = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest', 
    tools=tools_schema 
)



Tool schema defined for Gemini API using dictionary format.


Start a new chat session with this tool-enabled model, using the persona history.
# **Why:** Creates an interactive session that has both the desired persona and function-calling abilities.


In [68]:

chat_session_functions = model_with_tools.start_chat(
    history=initial_chat_history,
    enable_automatic_function_calling=False # Important: We will handle the calls
)
print("Model and chat session with function calling capabilities initialized.")

print("\nTesting function call for order status...")

Model and chat session with function calling capabilities initialized.

Testing function call for order status...


# **What:** Send a prompt designed to trigger the 'get_order_status' function.


In [69]:
prompt_order = "Can you tell me the status of my order ORD12345?"
print(f"User: {prompt_order}")



User: Can you tell me the status of my order ORD12345?


# **What:** Check if the model responded by requesting a function call.
# **Why:** This is the core logic for function calling interaction.


In [101]:
response_order = chat_session_functions.send_message(prompt_order)

# --- REFINED Manual Function Calling Handling (for response_order) ---
function_calls_to_process = []
final_response_text = None

try:
    # **What:** Safely access the parts of the *response_order*.
    if response_order.candidates and response_order.candidates[0].content.parts:
        print("--- Parsing response parts for function calls ---")
        for part in response_order.candidates[0].content.parts:
            if hasattr(part, 'function_call') and part.function_call:
                print(f"--- Found function call part: {part.function_call.name} ---")
                function_calls_to_process.append(part.function_call)
except Exception as e:
    print(f"Warning: Could not parse response parts for function calls - {e}")
    try:
        final_response_text = response_order.text
    except Exception:
        final_response_text = "(Error retrieving response text after parsing failure)"

if function_calls_to_process:
    print("Agent is requesting to use a tool(s)...")
    fc_list = function_calls_to_process
    function_results = []

    for fc in fc_list:
        function_name = fc.name
        function_args = {key: value for key, value in fc.args.items()}
        print(f"   Tool: {function_name}, Arguments: {function_args}")

        # **IMPORTANT**: This relies on 'available_functions' being defined earlier.
        if function_name in available_functions:
            function_to_call = available_functions[function_name]
            try:
                function_response_data = function_to_call(**function_args)
                function_results.append({
                    "function_response": {
                        "name": function_name,
                        "response": function_response_data # Send the raw dictionary result
                     }
                })
            except Exception as e:
                 print(f" Error executing function {function_name}: {e}")
                 function_results.append({
                     "function_response": {
                         "name": function_name,
                         "response": {"error": f"Failed to execute function {function_name}: {e}"}
                     }
                 })
        else:
            print(f" Error: Model requested an unknown function: {function_name}")
            function_results.append({
                "function_response": {
                    "name": function_name,
                    "response": {"error": f"Function '{function_name}' is not available."}
                }
            })

    print("--- Sending function results back to Agent ---")
    response_after_tools = chat_session_functions.send_message(function_results)

    try:
        final_response_text = response_after_tools.text
        print(f"Agent (after tool use): {final_response_text}")
    except Exception as e:
        print(f"Warning: Could not get text from response after tool use - {e}")

else:
    # No function calls detected in response_order
    try:
        final_response_text = response_order.text
        print(f"Agent (responded directly): {final_response_text}")
    except Exception as e:
        print(f"Warning: Could not get text from response - {e}")

print("-" * 30)


--- Parsing response parts for function calls ---
--- Found function call part: get_order_status ---
Agent is requesting to use a tool(s)...
   Tool: get_order_status, Arguments: {'order_id': 'ORD12345'}
--- MOCK FUNCTION CALLED: get_order_status(order_id='ORD12345') ---
--- MOCK FUNCTION RETURNED: {'status': 'Shipped', 'estimated_delivery': '2024-03-15', 'carrier': 'ExpressPost'} ---
--- Sending function results back to Agent ---
Agent (after tool use): Your order (ORD12345) has shipped via ExpressPost and is estimated to arrive on March 15, 2024.

------------------------------


**** Function Calling Test: Product Info****

In [102]:
# --- Function Calling Test: Product Info ---
print("\nTesting function call for product info...")
prompt_product = "How much are the gravity boots?"
print(f"User: {prompt_product}")


Testing function call for product info...
User: How much are the gravity boots?


In [104]:
response_product = chat_session_functions.send_message(prompt_product)

# --- REFINED Manual Function Calling Handling (for response_order) ---
function_calls_to_process = []
final_response_text = None

try:
    # **What:** Safely access the parts of the *response_order*.
    if response_product.candidates and response_product.candidates[0].content.parts:
        print("--- Parsing response parts for function calls ---")
        for part in response_product.candidates[0].content.parts:
            if hasattr(part, 'function_call') and part.function_call:
                print(f"--- Found function call part: {part.function_call.name} ---")
                function_calls_to_process.append(part.function_call)
except Exception as e:
    print(f"Warning: Could not parse response parts for function calls - {e}")
    try:
        final_response_text = response_product.text
    except Exception:
        final_response_text = "(Error retrieving response text after parsing failure)"

if function_calls_to_process:
    print("Agent is requesting to use a tool(s)...")
    fc_list = function_calls_to_process
    function_results = []

    for fc in fc_list:
        function_name = fc.name
        function_args = {key: value for key, value in fc.args.items()}
        print(f"   Tool: {function_name}, Arguments: {function_args}")

        # **IMPORTANT**: This relies on 'available_functions' being defined earlier.
        if function_name in available_functions:
            function_to_call = available_functions[function_name]
            try:
                function_response_data = function_to_call(**function_args)
                function_results.append({
                    "function_response": {
                        "name": function_name,
                        "response": function_response_data # Send the raw dictionary result
                     }
                })
            except Exception as e:
                 print(f" Error executing function {function_name}: {e}")
                 function_results.append({
                     "function_response": {
                         "name": function_name,
                         "response": {"error": f"Failed to execute function {function_name}: {e}"}
                     }
                 })
        else:
            print(f" Error: Model requested an unknown function: {function_name}")
            function_results.append({
                "function_response": {
                    "name": function_name,
                    "response": {"error": f"Function '{function_name}' is not available."}
                }
            })

    print("--- Sending function results back to Agent ---")
    response_after_tools = chat_session_functions.send_message(function_results)

    try:
        final_response_text = response_after_tools.text
        print(f"Agent (after tool use): {final_response_text}")
    except Exception as e:
        print(f"Warning: Could not get text from response after tool use - {e}")

else:
    # No function calls detected in response_order
    try:
        final_response_text = response_product.text
        print(f"Agent (responded directly): {final_response_text}")
    except Exception as e:
        print(f"Warning: Could not get text from response - {e}")

print("-" * 30)



--- Parsing response parts for function calls ---
--- Found function call part: find_product_info ---
Agent is requesting to use a tool(s)...
   Tool: find_product_info, Arguments: {'product_name': 'gravity boots'}
--- MOCK FUNCTION CALLED: find_product_info(product_name='gravity boots') ---
--- MOCK FUNCTION RETURNED: {'price': 89.5, 'availability': 'Ships in 3-5 days', 'features': ['Magnetic soles', 'Comfort fit', 'Sizes S/M/L']} ---
--- Sending function results back to Agent ---
Agent (after tool use): The gravity boots are currently priced at $89.50 and will ship in 3-5 business days.  They feature magnetic soles, a comfort fit, and are available in sizes small, medium, and large.

------------------------------


#  4. Capability Demo: Structured Output (JSON Mode)
#
* **Problem Solved:** Getting structured, predictable data from the AI for automated processing (e.g., routing tickets, analytics).
* **Gen AI Solution:** Instructing the model to output its response strictly in JSON format adhering to a specified schema.


In [105]:
print("\n--- Capability Demo: Structured Output (JSON Mode) ---")
print("Goal: Make the AI categorize a query and output the result as JSON.")


--- Capability Demo: Structured Output (JSON Mode) ---
Goal: Make the AI categorize a query and output the result as JSON.


# **What:** Define a prompt that explicitly asks for JSON output with a specific structure.
# **Why:** Guides the model to perform the analysis and format the result correctly.

In [107]:
json_prompt = """
Analyze the customer query below. Respond ONLY with a valid JSON object containing these keys:
- "category": (string) One of: "Order Inquiry", "Product Question", "Return Request", "Technical Support", "General Feedback", "Other"
- "keywords": (list of strings) Key terms or product names mentioned.
- "urgency": (string) One of: "Low", "Medium", "High".

Customer Query: "Hi, my Photon Blaster X1 seems to be malfunctioning, it's making a weird buzzing sound. Can I return it or get it fixed? My order was ORD11223."
"""

 **What:** Initialize a model specifically configured to output JSON.
# **Why:** The `response_mime_type` setting enforces JSON output format.
# **Note:** This will raise an error if the model name is invalid or API access fails.

In [108]:
model_json = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json" # Force JSON output
    )
)
print("Model for JSON output initialized.")

print("\nSending prompt for JSON categorization...")

Model for JSON output initialized.

Sending prompt for JSON categorization...


# **What:** Generate content using the JSON-specific model and prompt.

In [110]:
response_json = model_json.generate_content(json_prompt)

print("\nAgent (Raw JSON Output):")
print(response_json.text)


Agent (Raw JSON Output):
{"category": "Return Request", "keywords": ["Photon Blaster X1", "buzzing sound", "ORD11223"], "urgency": "Medium"}


# **What:** Print the raw text output from the model (expected to be a JSON string).

{"category": "Return Request", "keywords": ["Photon Blaster X1", "buzzing sound", "ORD11223"], "urgency": "Medium"}


 **What:** Attempt to parse the JSON string.
# **Why:** Verifies that the model returned valid JSON.

In [111]:
try:
    parsed_json = json.loads(response_json.text)
    print("\nParsed JSON object (for verification):")
    print(json.dumps(parsed_json, indent=2)) # Pretty-print
    print("\n Successfully received and parsed structured JSON output.")
except json.JSONDecodeError as e:
    # This try/except remains as it checks the *model's output*, not the setup.
    print(f"\n Error: Model response was not valid JSON. {e}")
    print(f"   Raw response received: {response_json.text}")
print("-" * 30)


Parsed JSON object (for verification):
{
  "category": "Return Request",
  "keywords": [
    "Photon Blaster X1",
    "buzzing sound",
    "ORD11223"
  ],
  "urgency": "Medium"
}

 Successfully received and parsed structured JSON output.
------------------------------


#  5. Simple Interactive Agent Loop (Combining Persona & Function Calling)
#
# **What:** A basic loop demonstrating the agent interacting with a user, using its persona and function-calling abilities.
# **Why:** Simulates a live chat session with the AI agent.
# **Note:** This section will fail if `chat_session_functions` was not successfully created in step 3 (e.g., due to API key issues).
#

In [112]:
print("\n--- Starting Interactive Customer Support Agent ---")
print("Using the agent with persona and function calling enabled.")
print("Ask about products (e.g., 'Photon Blaster X1', 'gravity boots')")
print("or check order status (e.g., 'ORD12345', 'ORD67890', 'ORD11223').")
print("Type 'quit' to exit.")


--- Starting Interactive Customer Support Agent ---
Using the agent with persona and function calling enabled.
Ask about products (e.g., 'Photon Blaster X1', 'gravity boots')
or check order status (e.g., 'ORD12345', 'ORD67890', 'ORD11223').
Type 'quit' to exit.


# The loop starts directly, assuming chat_session_functions exists.


In [118]:
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Agent: Thank you for contacting GadgetGalaxy support. Goodbye!")
        break
    if not user_input:
        continue

   # print("--- Sending message to Agent ---") # Debug print
    initial_response = chat_session_functions.send_message(user_input)
    
    function_calls_to_process = []
    final_response_text = None

    try:
        if initial_response.candidates and initial_response.candidates[0].content.parts:
          #  print("--- Parsing response parts for function calls ---")
            for part in initial_response.candidates[0].content.parts:
                if hasattr(part, 'function_call') and part.function_call:
                    print(f"--- Found function call part: {part.function_call.name} ---")
                    function_calls_to_process.append(part.function_call)
    except Exception as e:
        print(f"Warning: Could not parse initial response parts for function calls - {e}")
        try:
            final_response_text = initial_response.text
        except Exception:
            final_response_text = "(Error retrieving response text after parsing failure)"

    if function_calls_to_process:
      #  print("Agent is requesting to use a tool(s)...")
        fc_list = function_calls_to_process
        function_results = []

        for fc in fc_list:
            function_name = fc.name
            function_args = {key: value for key, value in fc.args.items()}
            print(f"   Tool: {function_name}, Arguments: {function_args}")

            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                try:
                    function_response_data = function_to_call(**function_args)
                    function_results.append({
                        "function_response": {
                            "name": function_name,
                            "response": function_response_data # Send the raw dictionary result
                         }
                    })
                except Exception as e:
                     print(f" Error executing function {function_name}: {e}")
                     function_results.append({
                         "function_response": {
                             "name": function_name,
                             "response": {"error": f"Failed to execute function {function_name}: {e}"}
                         }
                     })
            else:
                print(f"Error: Model requested an unknown function: {function_name}")
                function_results.append({
                    "function_response": {
                        "name": function_name,
                        "response": {"error": f"Function '{function_name}' is not available."}
                    }
                })

     #   print("--- Sending function results back to Agent ---")
        response_after_tools = interactive_chat.send_message(function_results)

        try:
            final_response_text = response_after_tools.text
            print(f"Agent: {final_response_text}")
        except Exception as e:
            print(f"Warning: Could not get text from response after tool use - {e}")

    else:
        # No function calls detected in initial_response
        try:
            final_response_text = initial_response.text
            print(f"Agent : {final_response_text}")
        except Exception as e:
            print(f"Warning: Could not get text from initial response - {e}")



You:  hi


Agent : Hi there! Welcome to GadgetGalaxy support.  How can I help you today?



You:  quit


Agent: Thank you for contacting GadgetGalaxy support. Goodbye!
